<a href="https://colab.research.google.com/github/AlySerry0/CSEN903-Airline-Team-133/blob/main/Airline_133.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AlySerry0/CSEN903-Airline-RawData.git

raw_data_dir = '/content/CSEN903-Airline-RawData'

Cloning into 'CSEN903-Airline-RawData'...


In [2]:
!pip install vaderSentiment
import pandas as pd
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

ModuleNotFoundError: No module named 'pandas'

In [ ]:
AirlineScrappedReview = pd.read_csv(f'{raw_data_dir}/AirlineScrappedReview_Cleaned.csv')
AirlineScrappedReview.info()

In [ ]:
AirlineScrappedReview.head()

In [ ]:
AirlineScrappedReview.isnull().sum()

In [ ]:
AirlineScrappedReview.describe(include='all')

In [ ]:
columns_to_drop = [
    'Flying_Date', 'Layover_Route',        # Too many missing values
    'Passanger_Name', 'Review_title',       # Unneeded identifiers
    'Start_Latitude', 'Start_Longitude',    # Redundant location data
    'Start_Address', 'End_Latitude',
    'End_Longitude', 'End_Address',
    'Start_Location', 'End_Location'      # Redundant to 'Route'
]
AirlineScrappedReview = AirlineScrappedReview.drop(columns=columns_to_drop)
# AirlineScrappedReview = AirlineScrappedReview.dropna()

In [ ]:
AirlineScrappedReview['Satisfaction'] = (AirlineScrappedReview['Rating'] >= 5).astype(int)
sia = SentimentIntensityAnalyzer()
AirlineScrappedReview['Sentiment_Score'] = AirlineScrappedReview['Review_content'].apply(
    lambda review: sia.polarity_scores(review)['compound']
)

In [ ]:
# --- 5. Encode Binary 'Verified' Column ---
# Map 'Trip Verified' to 1 and 'Not Verified' to 0
AirlineScrappedReview['Verified'] = AirlineScrappedReview['Verified'].map({
    'Trip Verified': 1,
    'Not Verified': 0
})

# --- 6. One-Hot Encode 'Traveller_Type' and 'Class' ---
AirlineScrappedReview = pd.get_dummies(AirlineScrappedReview,
                            columns=['Traveller_Type', 'Class'],
                            drop_first=True)

AirlineScrappedReview = AirlineScrappedReview.drop_duplicates()

# Display the final prepped DataFrame
print("\n--- Final Prepped DataFrame ---")
AirlineScrappedReview.info()

In [ ]:
Customer_comment = pd.read_csv(f'{raw_data_dir}/Customer_comment.csv')

In [ ]:
Customer_comment.info()

In [ ]:
Customer_comment.head()

In [ ]:
Customer_comment.isnull().sum()

In [ ]:
Customer_comment.describe(include='all')

In [ ]:
print("\n--- Unique Values in Key Columns ---")
cols_to_check = ['arrival_delay_group', 'departure_delay_group', 'entity',
                 'seat_factor_band', 'loyalty_program_level', 'fleet_usage', 'response_group', 'sentiments']

for col in cols_to_check:
    print(f"Column '{col}': {Customer_comment[col].unique()}")

In [ ]:
# --- 1. Drop Unnecessary/Redundant Columns ---
columns_to_drop = [
    'Unnamed: 0',         # Duplicate index
    'ques_verbatim_text', # Only has one unique value, no info
    'transformed_text'    # Incomplete, and we have the raw 'verbatim_text'
]
Customer_comment = Customer_comment.drop(columns=columns_to_drop)

# --- 2. Handle Missing Data ---
# Fill missing loyalty levels with 'None'
Customer_comment['loyalty_program_level'] = Customer_comment['loyalty_program_level'].fillna('None')

# --- 3. Convert Data Types ---
# Convert date column to datetime objects
Customer_comment['scheduled_departure_date'] = pd.to_datetime(Customer_comment['scheduled_departure_date'])

# --- 4. Encode Binary Categorical Columns ---
# We'll map the "negative" or "delayed" state to 1 and the "positive" state to 0

Customer_comment['arrival_delay_group'] = Customer_comment['arrival_delay_group'].map({
    'Delayed': 1,
    'Early & Ontime': 0
})

Customer_comment['departure_delay_group'] = Customer_comment['departure_delay_group'].map({
    'Delayed': 1,
    'Early & Ontime': 0
})

Customer_comment['sentiments'] = Customer_comment['sentiments'].map({
    'Negative': 1,
    'Neutral': 0
})

# --- 5. Encode Ordinal 'seat_factor_band' Column ---
band_mapping = {
    '0 to 70': 0,
    '70+': 1,
    '80+': 2,
    '90+': 3
}

Customer_comment['seat_factor_band'] = Customer_comment['seat_factor_band'].map(band_mapping)

Customer_comment = Customer_comment.drop_duplicates()


Customer_comment.info()
Customer_comment.head()